# Spanish Voice Cloning with Tortoise TTS

<a href="https://colab.research.google.com/github/juanvolpe/voiceJuan/blob/main/colab_spanish_tts.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook will help you:
1. Set up the Spanish voice cloning system
2. Upload your voice samples
3. Generate Spanish speech with your voice

Let us begin with the setup:

In [ ]:
# Check version and setup
import requests

def get_latest_commit():
    repo_api = "https://api.github.com/repos/juanvolpe/voiceJuan/commits/main"
    response = requests.get(repo_api)
    if response.status_code == 200:
        return response.json()['sha']
    return None

# Clone repository
!git clone https://github.com/juanvolpe/voiceJuan.git
%cd voiceJuan

# Check version
latest_commit = get_latest_commit()
if latest_commit:
    print(f"\nLatest GitHub commit: {latest_commit}")
    print("\nIf you want the latest version:")
    print("1. File → Reset runtime")
    print("2. Runtime → Run all")

# Install dependencies
!pip install -r requirements.txt

## Upload Voice Samples

Please prepare your WAV files with these requirements:
- Clear Spanish speech
- WAV format
- Good quality audio

Use the "Choose Files" button below to upload your samples:

In [ ]:
from google.colab import files
import os

# Create directories
!mkdir -p tortoise/voices/juan_es/samples

# Upload interface
print("Please upload your WAV files...")
uploaded = files.upload()

# Save files
for filename in uploaded.keys():
    if filename.endswith('.wav'):
        path = f'tortoise/voices/juan_es/samples/{filename}'
        with open(path, 'wb') as f:
            f.write(uploaded[filename])
        print(f'Saved {filename}')

# List all uploaded samples
print("\nUploaded voice samples:")
!ls tortoise/voices/juan_es/samples/

## Generate Speech

Ready to generate speech with your voice samples! You will have two options:
1. Use existing voice cache (faster)
2. Reprocess voice samples (choose this if you added new samples)

Run the code below to begin:

In [ ]:
from spanish_tortoise import SpanishTTS
from IPython.display import Audio

# Initialize TTS
tts = SpanishTTS()  # Will ask about cache usage

# Get text input
text = input("Enter Spanish text: ")

# Available presets
presets = ['ultra_fast', 'fast', 'standard', 'high_quality']
print("\nAvailable quality presets:")
for i, p in enumerate(presets, 1):
    print(f"{i}. {p}")

# Get preset choice
while True:
    choice = input("\nSelect quality (1-4) [default=2]: ").strip()
    if not choice:
        preset = 'fast'
        break
    try:
        idx = int(choice) - 1
        if 0 <= idx < len(presets):
            preset = presets[idx]
            break
    except ValueError:
        pass
    print("Please enter a number between 1 and 4")

# Generate speech
output_file = tts.generate_speech(text, preset=preset)

# Play the generated audio
Audio(output_file)

## Download Generated Audio

Click below to save the generated audio file to your computer:

In [ ]:
files.download(output_file)